In [2]:
import numpy as np 
import pandas as pd 
import warnings

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

warnings.filterwarnings("ignore",  category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning,)
warnings.filterwarnings("ignore", category=UserWarning)


/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [3]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [4]:
excluded_cols = ['id', 'Personality']
all_columns = train_df.columns
# Przetwarzanie pozostałych kolumn
for col in all_columns:
    if col not in excluded_cols:
        train_df[col + '_MISS'] = train_df[col].notna().astype(int)
        test_df[col + '_MISS'] = test_df[col].notna().astype(int)


columns = ['Time_spent_Alone_MISS','Stage_fear_MISS', 'Social_event_attendance_MISS', 'Going_outside_MISS','Drained_after_socializing_MISS', 
           'Friends_circle_size_MISS','Post_frequency_MISS']
train_df['not_MISS_total'] = train_df[columns].sum(axis=1)
test_df['not_MISS_total'] = test_df[columns].sum(axis=1)

train_df['Stage_fear'] = train_df['Stage_fear'].mask(train_df['Stage_fear'].isna() & train_df['Drained_after_socializing']
                                            .notna(), train_df['Drained_after_socializing'])
train_df['Drained_after_socializing'] = train_df['Drained_after_socializing'].mask(train_df['Drained_after_socializing']
                                            .isna() & train_df['Stage_fear'].notna(), train_df['Stage_fear'])

test_df['Stage_fear'] = test_df['Stage_fear'].mask(test_df['Stage_fear'].isna() & test_df['Drained_after_socializing']
                                            .notna(), test_df['Drained_after_socializing'])
test_df['Drained_after_socializing'] = test_df['Drained_after_socializing'].mask(test_df['Drained_after_socializing']
                                            .isna() & test_df['Stage_fear'].notna(), test_df['Stage_fear'])

train_df['Time_Alone_dev_Outside'] = train_df['Time_spent_Alone'] / train_df['Going_outside']
train_df['Time_Alone_dev_Outside']=train_df['Time_Alone_dev_Outside'].round(2).astype(float)
def Time_Alone_dev_Outside (x):
    try:
        x=float(x)
        if x <= 1:
            return 0
        elif x > 1 and x < 2:
            return 1
        elif x >= 2 and x < 100:
            return 2
        else:
            return 3
    except ValueError:
        return 3

train_df['Time_Alone_dev_Outside']=train_df['Time_Alone_dev_Outside'].apply(Time_Alone_dev_Outside).astype('Int64')

train_df['Social_dev_Post'] = train_df['Social_event_attendance'] / train_df['Post_frequency']
train_df['Social_dev_Post']=train_df['Social_dev_Post'].round(2).astype(float)

def Social_dev_Post (x):
    try:
        x=float(x)
        if x == 0:
            return 0
        elif x > 0 and x < 0.33:
            return 1
        elif x == 0.33:
            return 2
        elif x > 0.33 and x < 0.5:
            return 3
        elif x == 0.5:
            return 4
        elif x > 0.5 and x < 0.67:
            return 5
        elif x == 0.67:
            return 6
        elif x > 0.67 and x < 1:
            return 7
        elif x == 1:
            return 8
        elif x > 1 and x < 1.5:
            return 9
        elif x == 1.5:
            return 10
        elif x > 1.5 and x < 2:
            return 11
        elif x == 2:
            return 12
        elif x > 2 and x < 3:
            return 13
        elif x == 3:
            return 14
        elif x > 3 and x < 100:
            return 15
        else:
            return 16
    except ValueError:
        return 16

train_df['Social_dev_Post']=train_df['Social_dev_Post'].apply(Social_dev_Post).astype('Int64')


train_df['Outside_mult_Friends'] = train_df['Going_outside'] * train_df['Friends_circle_size']
def Outside_mult_Friends (x):
    try:
        x=float(x)
        if x <= 11:
            return 0
        elif x > 11 and x <= 15:
            return 1
        elif x > 15 and x < 400:
            return 2
        else:
            return 2
    except ValueError:
        return 2

train_df['Outside_mult_Friends']=train_df['Outside_mult_Friends'].apply(Outside_mult_Friends).astype('Int64')

In [5]:
cols_to_convert = ['Time_spent_Alone', 'Social_event_attendance','Going_outside',  'Friends_circle_size','Post_frequency']
train_df[cols_to_convert] = train_df[cols_to_convert].astype('Int64')
test_df[cols_to_convert] = test_df[cols_to_convert].astype('Int64')

In [6]:
# Uzupełnienie danych Yes - Yes // No- No
train_df['Stage_fear'] = train_df['Stage_fear'].mask(train_df['Stage_fear'].isna() & train_df['Drained_after_socializing']
                                            .notna(), train_df['Drained_after_socializing'])
train_df['Drained_after_socializing'] = train_df['Drained_after_socializing'].mask(train_df['Drained_after_socializing']
                                            .isna() & train_df['Stage_fear'].notna(), train_df['Stage_fear'])

test_df['Stage_fear'] = test_df['Stage_fear'].mask(test_df['Stage_fear'].isna() & test_df['Drained_after_socializing']
                                            .notna(), test_df['Drained_after_socializing'])
test_df['Drained_after_socializing'] = test_df['Drained_after_socializing'].mask(test_df['Drained_after_socializing']
                                            .isna() & test_df['Stage_fear'].notna(), test_df['Stage_fear'])

In [7]:
train_df['Time_Alone_dev_Outside'] = train_df['Time_spent_Alone'] / train_df['Going_outside']
train_df['Time_Alone_dev_Outside']=train_df['Time_Alone_dev_Outside'].round(2).astype(float)
def Time_Alone_dev_Outside (x):
    try:
        x=float(x)
        if x <= 1:
            return 0
        elif x > 1 and x < 2:
            return 1
        elif x >= 2 and x < 100:
            return 2
        else:
            return 3
    except ValueError:
        return 3

train_df['Time_Alone_dev_Outside']=train_df['Time_Alone_dev_Outside'].apply(Time_Alone_dev_Outside).astype('Int64')

test_df['Time_Alone_dev_Outside'] = test_df['Time_spent_Alone'] / test_df['Going_outside']
test_df['Time_Alone_dev_Outside']=test_df['Time_Alone_dev_Outside'].round(2).astype(float)
test_df['Time_Alone_dev_Outside']=test_df['Time_Alone_dev_Outside'].apply(Time_Alone_dev_Outside).astype('Int64')

In [8]:
train_df['Social_dev_Post'] = train_df['Social_event_attendance'] / train_df['Post_frequency']
train_df['Social_dev_Post']=train_df['Social_dev_Post'].round(2).astype(float)

def Social_dev_Post (x):
    try:
        x=float(x)
        if x == 0:
            return 0
        elif x > 0 and x < 0.33:
            return 1
        elif x == 0.33:
            return 2
        elif x > 0.33 and x < 0.5:
            return 1
        elif x == 0.5:
            return 4
        elif x > 0.5 and x < 0.67:
            return 1
        elif x == 0.67:
            return 4
        elif x > 0.67 and x < 1:
            return 1
        elif x == 1:
            return 4
        elif x > 1 and x < 1.5:
            return 1
        elif x == 1.5:
            return 4
        elif x > 1.5 and x < 2:
            return 1
        elif x == 2:
            return 4
        elif x > 2 and x < 3:
            return 1
        elif x == 3:
            return 2
        elif x > 3 and x < 100:
            return 1
        else:
            return 0
    except ValueError:
        return 0

train_df['Social_dev_Post']=train_df['Social_dev_Post'].apply(Social_dev_Post).astype('Int64')


test_df['Social_dev_Post'] = test_df['Social_event_attendance'] / test_df['Post_frequency']
test_df['Social_dev_Post']=test_df['Social_dev_Post'].round(2).astype(float)
test_df['Social_dev_Post']=test_df['Social_dev_Post'].apply(Social_dev_Post).astype('Int64')

In [9]:
train_df['Outside_mult_Friends'] = train_df['Going_outside'] * train_df['Friends_circle_size']
def Outside_mult_Friends (x):
    try:
        x=float(x)
        if x <= 11:
            return 0
        elif x > 11 and x <= 15:
            return 1
        elif x > 15 and x < 400:
            return 2
        else:
            return 2
    except ValueError:
        return 2

train_df['Outside_mult_Friends']=train_df['Outside_mult_Friends'].apply(Outside_mult_Friends).astype('Int64')

test_df['Outside_mult_Friends'] = test_df['Going_outside'] * test_df['Friends_circle_size']
test_df['Outside_mult_Friends']=test_df['Outside_mult_Friends'].apply(Outside_mult_Friends).astype('Int64')

In [10]:
cat_cols = ['Stage_fear','Drained_after_socializing']  # <-- tu wpisz swoje kolumny kategoryczne

train_df[cat_cols]=train_df[cat_cols].fillna('Missing').astype(str)
test_df[cat_cols] = test_df[cat_cols].fillna('Missing').astype(str)


In [11]:
train_df.drop(['Time_spent_Alone_MISS','Stage_fear_MISS', 'Social_event_attendance_MISS', 'Going_outside_MISS',
              'Drained_after_socializing_MISS', 'Friends_circle_size_MISS','Post_frequency_MISS'], axis=1, inplace=True)
test_df.drop(['Time_spent_Alone_MISS','Stage_fear_MISS', 'Social_event_attendance_MISS', 'Going_outside_MISS',
              'Drained_after_socializing_MISS', 'Friends_circle_size_MISS','Post_frequency_MISS'], axis=1, inplace=True)

In [12]:
train_df.columns

Index(['id', 'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Personality', 'not_MISS_total',
       'Time_Alone_dev_Outside', 'Social_dev_Post', 'Outside_mult_Friends'],
      dtype='object')


train_df['Sum'] = train_df['Social_event_attendance']+train_df['Going_outside']+train_df['Friends_circle_size']+train_df['Post_frequency']
test_df['Sum'] = test_df['Social_event_attendance']+test_df['Going_outside']+test_df['Friends_circle_size']+test_df['Post_frequency']


In [13]:
from catboost import CatBoostClassifier

cat_features = ['Stage_fear','Drained_after_socializing', 'not_MISS_total', 'Outside_mult_Friends', 'Social_dev_Post','Time_Alone_dev_Outside']# 'Time_spent_Alone_MISS',
       #'Stage_fear_MISS', 'Social_event_attendance_MISS', 'Going_outside_MISS',
       #'Drained_after_socializing_MISS', 'Friends_circle_size_MISS',
       #'Post_frequency_MISS']

model = CatBoostClassifier(
    cat_features=cat_features,
    iterations=2000,
    learning_rate=0.01, #było 0.05  - result 09740
    depth=6,
    l2_leaf_reg = 5,
    eval_metric = 'Logloss',
    verbose=100,
    random_seed=42
)

# Zakładamy, że target to np. 'label'
model.fit(train_df.drop(['id','Personality'], axis=1), train_df['Personality'])

# Predykcja
preds = model.predict(test_df.drop('id', axis=1))

submission = pd.DataFrame({
    'id': test_df['id'],
    'Personality': preds
})


submission.to_csv("submission.csv", index=False)

print('Sukcesss!!!!')

0:	learn: 0.6762815	total: 81.5ms	remaining: 2m 42s
100:	learn: 0.1653446	total: 1.94s	remaining: 36.4s
200:	learn: 0.1311767	total: 3.72s	remaining: 33.3s
300:	learn: 0.1261170	total: 5.5s	remaining: 31s
400:	learn: 0.1244619	total: 7.39s	remaining: 29.5s
500:	learn: 0.1234718	total: 9.55s	remaining: 28.6s
600:	learn: 0.1227477	total: 11.8s	remaining: 27.4s


KeyboardInterrupt: 